# Using SimpliVity REST API with Python

- This sample Python code performs user authentication.
- It then obtains the token from SimpliVity through a POST to the SimpliVity OVC or MVA
- Additionally, this sample code will perform the clone from tinyvm32_0000_0000_0000_0000 to *studentX_clone*

<div class="alert alert-block alert-danger">
<b>READ FIRST</b> Please replace XX with the number XX from username studentXX for the clone operation to succeed(critical).
</div>

This sample use modules provided by Phyton 3.7

In [ ]:
import pycurl
import json
import base64
import urllib
from io import BytesIO
import requests
import sys, getopt

### Reminder to change the XX to the number assigned to you as part of the studentXX

In [ ]:
#IP or Hostname of OVC or MVA
ovc_ip = '16.31.87.101'

#Username and Password
ovc_username = 'root'
ovc_password = 'vmware'

# VM to clone is one of the demo VM
sourcevmName = 'tinyvm32_0000_0000_0000_0000'
clonevmName = "student$$I_clone"

### Function to perform POST to get the Access token

In [ ]:
#Fuction using pyCurl to post to SVT REST using url encoded data

def post_SVTREST(url, data, header):
   rdata = BytesIO()
   pdata = urllib.parse.urlencode(data)
   c = pycurl.Curl()
   c.setopt(pycurl.SSL_VERIFYPEER, 0)
   c.setopt(pycurl.SSL_VERIFYHOST, 0)
   c.setopt(pycurl.URL, url)
   c.setopt(pycurl.HTTPHEADER, header)
   c.setopt(c.WRITEFUNCTION, rdata.write)
   c.setopt(pycurl.POST, 1)
   c.setopt(pycurl.POSTFIELDS, pdata)
   c.perform()
   c.close()
   jdata = json.loads(rdata.getvalue())
   return (jdata)

In [ ]:
#Get Access Token
url = 'https://'+ ovc_ip +'/api/oauth/token'
data = {'username': ovc_username,'password': ovc_password,'grant_type':'password'}
svtuser = bytes('simplivity:','utf8')
svtauth = bytes('Authorization:  Basic ','utf8') + base64.standard_b64encode(svtuser)
htoken = ['Accept: application/json', svtauth ]
svttoken = post_SVTREST(url, data, htoken)
print ('SVT token', json.dumps(svttoken, indent=4, sort_keys=True))
accesstoken = 'Authorization: Bearer ' + svttoken["access_token"]
print ('SVT access token', accesstoken)

### Function to perform GET to get list of objects

In [ ]:
#Funtion using pyCurl to get from SVT REST

def get_SVTREST(url, header):
   rdata = BytesIO()
   c = pycurl.Curl()
   c.setopt(pycurl.SSL_VERIFYPEER, 0)
   c.setopt(pycurl.SSL_VERIFYHOST, 0)
   c.setopt(pycurl.URL, url)
   c.setopt(pycurl.HTTPHEADER, header)
   c.setopt(c.WRITEFUNCTION, rdata.write)
   c.perform()
   c.close()
   jdata = json.loads(rdata.getvalue())
   return (jdata)

In [ ]:
#Get VM Id of Source VM

htoken = ['Accept: application/json', accesstoken]
url = 'https://'+ ovc_ip +'/api/virtual_machines?show_optional_fields=false&name=' +  sourcevmName
SVTvms = get_SVTREST(url, htoken)
vm_id = SVTvms['virtual_machines'][0]['id']

In [ ]:
SVTvms

In [ ]:
vm_id

### Function to perform cloning 

In [ ]:
#Fuction using pyCurl to post to SVT REST using json data

def jpost_SVTREST(url, data, header):
   rdata = BytesIO()
   c = pycurl.Curl()
   c.setopt(pycurl.SSL_VERIFYPEER, 0)
   c.setopt(pycurl.SSL_VERIFYHOST, 0)
   c.setopt(pycurl.URL, url)
   c.setopt(pycurl.HTTPHEADER, header)
   c.setopt(c.WRITEFUNCTION, rdata.write)
   c.setopt(pycurl.POST, 1)
   c.setopt(pycurl.POSTFIELDS, data)
   c.perform()
   c.close()
   jdata = json.loads(rdata.getvalue())
   return (jdata)

In [ ]:
#Clone Source VM to Clone

url = 'https://'+ ovc_ip +'/api/virtual_machines/'+ vm_id +'/clone'
data = {"app_consistent": "false", "virtual_machine_name": clonevmName}
sdata = json.dumps(data)
htoken = ['Accept: application/json', 'Content-Type: application/vnd.simplivity.v1+json', accesstoken ]
vmclone = jpost_SVTREST(url, sdata, htoken)

In [ ]:
vmclone


#### Conclusion: Python provides comprehensive methods to perform extensive SimpliVity REST API operation

# Continue now to Lab 4: Python Module Example
Description: This will allow us to use the released Python module to support complex operations.

* [Lab 4](PythonSDKExample.ipynb)


# Or return back to the Introduction Notebook
Description: This will allow you to try other notebooks.

* [Introduction](Introduction.ipynb)